In [67]:
# add autoreload
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import gc
from collections import defaultdict

from typing import List, Tuple, Dict, Union

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
import torch

In [69]:
from setfit import SetFitModel
from setfit import Trainer, TrainingArguments
from sentence_transformers.losses import CosineSimilarityLoss
from sentence_transformers import SentenceTransformer

from datasets import Dataset, DatasetDict
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score

#from transformers import TrainingArguments, Trainer
from transformers import pipeline
from transformers import DataCollatorWithPadding
from transformers import EvalPrediction

from torchinfo import summary
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import evaluate

from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer

import benedict
import random

In [70]:
from sklearn.svm import LinearSVC, SVC

In [71]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [72]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("There are %d GPU(s) available." % torch.cuda.device_count())
    print("We will use the GPU:", torch.cuda.get_device_name(0))
    print(f"Memory Allocated: {torch.cuda.memory_allocated()}")
    print(f"Max memory Allocated: {torch.cuda.max_memory_allocated()}") 
    print(f"Memory reserved: {torch.cuda.memory_reserved()}")
    print(f"Max memory reserved: {torch.cuda.max_memory_reserved()}")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA RTX 4000 Ada Generation
Memory Allocated: 3415247872
Max memory Allocated: 7992522752
Memory reserved: 12106858496
Max memory reserved: 12106858496


In [73]:
deabbreviate = False
filter_reports = True
Class = 'diastolic_dysfunction' # lv_dil, rv_dil, pe, aortic_regurgitation, diastolic_dysfunction, lv_syst_func, aortic_stenosis, rv_syst_func, mitral_regurgitation,tricuspid_regurgitation,wma
FLAG_TERMS = ['uitslag zie medische status', 'zie status', 'zie verslag status', 'slecht echovenster', 'echo overwegen', 'ge echo',
              'geen echovenster', 'geen beoordeelbaar echo', 'geen beoordeelbare echo', 'verslag op ic']
SAVE_TERMS = ['goed', 'geen', 'normaal', 'normale']
use_multilabel = False
reduce_labels = False
num_epochs = 3
NUM_SAMPLES = 500

#body = "FremyCompany/BioLORD-2023-M"
body = "NetherlandsForensicInstitute/robbert-2022-dutch-sentence-transformers"

MULTILABELS = {'Mild': ['Mild', 'Present'], 
               'Severe': ['Severe', 'Present'],
               'Moderate': ['Moderate', 'Present'],
               'Normal': ['Normal'],
               'No label': ['No label'],
               'Present': ['Present'],
               }

Target_remapper = {'No label': 'No label', 
                   'Normal': 'Normal',
                   'Mild': 'Not normal',
                   'Moderate': 'Not normal',
                   'Severe': 'Not normal',
                   'Present':  'Not normal'
                   }

#Target_remapper = {'No label': 'Normal', 
#                   'Normal': 'Normal',
#                   'Mild': 'Not normal',
#                   'Moderate': 'Not normal',
#                   'Severe': 'Not normal',
#                   'Present':  'Not normal'
#                   }

In [74]:
#Add the src folder to the path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
import deabber, echo_utils

In [75]:
if deabbreviate:
    ABBREVIATIONS = benedict.benedict("../assets/abbreviations.yml")

In [76]:
plt.style.use('ggplot')
def plot_history(history, val=0):
    acc = history.history['accuracy']
    if val == 1:
        val_acc = history.history['val_accuracy'] # we can add a validation set in our fit function with nn
    loss = history.history['loss']
    if val == 1:
        val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training accuracy')
    if val == 1:
        plt.plot(x, val_acc, 'r', label='Validation accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.title('Accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    if val == 1:
        plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.title('Loss')
    plt.legend()

In [77]:
def compute_metrics_binomial(logits_and_labels, averaging='macro'):
  logits, labels = logits_and_labels
  predictions = np.argmax(logits, axis=-1)
  acc = np.mean(predictions == labels)
  f1 = f1_score(labels, predictions, average = averaging)
  prec = precision_score(labels, predictions, average = averaging)
  rec = recall_score(labels, predictions, average = averaging)
  return {
          'accuracy': acc, 
          'f1_score': f1,
          'precision': prec,
          'recal': rec
          }

def compute_metrics(logits_and_labels):
  logits, labels = logits_and_labels
  predictions = np.argmax(logits, axis=-1)
  
  f1_micro = f1_score(labels, predictions, average = 'micro')
  f1_macro = f1_score(labels, predictions, average = 'macro')
  
  precision_micro = precision_score(labels, predictions, average = 'micro')
  precision_macro = precision_score(labels, predictions, average = 'macro')

  recall_micro = recall_score(labels, predictions, average = 'micro')
  recall_macro = recall_score(labels, predictions, average = 'macro')
  
  return {'precision_micro': precision_micro, 'precision_macro': precision_macro,
          'f1_score_micro': f1_micro, 'f1_score_macro': f1_macro, 
          'recall_micro': recall_micro, 'recall_macro': recall_macro}
  
def extract_metrics_agg(probas, labels, thresholds, average = 'macro'):
  metrics = []
  for t in thresholds:
    preds = probas.argmax(axis=1)
    preds = np.where(probas.max(axis=1)<t, -1, preds)
    
    idx_check = np.argwhere(preds > -1)[:,0]
    idx_ignore = np.argwhere(preds == -1)[:,0]
        
    precision = precision_score(labels[idx_check], preds[idx_check], average = average)
    recall = recall_score(labels[idx_check], preds[idx_check], average =average)
    f1 = f1_score(labels[idx_check], preds[idx_check], average = average)
    accuracy = accuracy_score(labels, preds)
    
    metrics.append([precision, recall, f1, accuracy, len(idx_check) / len(labels)])
  return np.array(metrics)


def extract_confusion_matrix(probas, labels, thresholds):
  # confusion_class_0 = [(threshold, TP, FP, TN, FN)]
  
  res_dict = defaultdict(tuple)
    
  for t in thresholds:
    preds = probas.argmax(axis=1)
    preds = np.where(probas.max(axis=1)<t, -1, preds)    
    idx_check = np.argwhere(preds > -1)[:,0]
    #
    for _class in labels.unique():
      TP = np.sum(np.logical_and(preds[idx_check] == 0, labels[idx_check] == 0))
      TN = np.sum(np.logical_and(preds[idx_check] != 0, labels[idx_check] != 0))
      FP = np.sum(np.logical_and(preds[idx_check] == 0, labels[idx_check] != 0))
      FN = np.sum(np.logical_and(preds[idx_check] != 0, labels[idx_check] == 0))
      res_dict[_class].append((t, TP, FP, TN, FN))
  return res_dict


def plot_AUROC(ax, probas, labels, class_idx = 0, label = None):
  from sklearn.metrics import roc_curve, auc
  fpr, tpr, _ = roc_curve((labels==class_idx).astype('int'),
                          probas[:, class_idx])
  roc_auc = auc(fpr, tpr)
  ax.plot(fpr, tpr, lw=2, label=f'{label} (area = {roc_auc:.2f})')
  ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
  ax.set_xlim([0.0, 1.0])
  ax.set_ylim([0.0, 1.05])
  ax.set_xlabel('False Positive Rate')
  ax.set_ylabel('True Positive Rate')
  ax.set_title(f'ROC curve')
  ax.legend(loc="lower right")
  return ax

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
'''
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    
    
    f1_macro = f1_score(y_true=y_true, y_pred=y_pred, average='macro')
    f1_weighted = f1_score(y_true=y_true, y_pred=y_pred, average='weighted')
    prec_macro = precision_score(y_true=y_true, y_pred=y_pred, average='macro')
    prec_weighted = precision_score(y_true=y_true, y_pred=y_pred, average='weighted')
    recall_macro = recall_score(y_true=y_true, y_pred=y_pred, average='macro')
    recall_weighted = recall_score(y_true=y_true, y_pred=y_pred, average='weighted')    
    
    try:
        roc_auc_weighted = roc_auc_score(y_true, probs, average = 'weighted')
        roc_auc_macro = roc_auc_score(y_true, probs, average = 'macro')
    except:
        roc_auc_weighted = None
        roc_auc_macro = None
    
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1_macro': f1_macro,
               'f1_weighted': f1_weighted,
               'prec_macro': prec_macro,
               'prec_weighted': prec_weighted,
               'recall_macro': recall_macro,
               'recall_weighted': recall_weighted,
               'roc_auc_macro': roc_auc_macro,
               'roc_auc_weighted': roc_auc_weighted,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result
'''


"\ndef multi_label_metrics(predictions, labels, threshold=0.5):\n    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)\n    sigmoid = torch.nn.Sigmoid()\n    probs = sigmoid(torch.Tensor(predictions))\n    # next, use threshold to turn them into integer predictions\n    y_pred = np.zeros(probs.shape)\n    y_pred[np.where(probs >= threshold)] = 1\n    # finally, compute metrics\n    y_true = labels\n    \n    \n    f1_macro = f1_score(y_true=y_true, y_pred=y_pred, average='macro')\n    f1_weighted = f1_score(y_true=y_true, y_pred=y_pred, average='weighted')\n    prec_macro = precision_score(y_true=y_true, y_pred=y_pred, average='macro')\n    prec_weighted = precision_score(y_true=y_true, y_pred=y_pred, average='weighted')\n    recall_macro = recall_score(y_true=y_true, y_pred=y_pred, average='macro')\n    recall_weighted = recall_score(y_true=y_true, y_pred=y_pred, average='weighted')    \n    \n    try:\n        roc_auc_weighted = roc_auc_score(y_true, p

In [78]:
def one_hot_encode(labels, num_classes):
    # labels should be a list or a 1D tensor
    one_hot = torch.zeros((len(labels), num_classes))
    rows = torch.arange(len(labels))
    one_hot[rows, labels] = 1
    return one_hot

def _mlabel_tuple_creator(x: List[int],
                          multilabels:Dict[int,List[int]],
                          num_classes: int=None)\
                          ->List[Tuple[int,...]]:
                              
    res = [(_sc for _sc in multilabels[sc]) for sc in x]
    return res

def multi_hot_encoding(x: List[int], 
                       multilabels: Union[Dict[int,List[int]], None]=None,
                       num_classes: int=None)\
                           ->torch.Tensor:    
    if multilabels is None:
        return one_hot_encode(x, num_classes=num_classes)
    else:
        return torch.Tensor(MultiLabelBinarizer(classes=range(num_classes))\
                    .fit_transform(_mlabel_tuple_creator(x,multilabels)))

# Load documents

In [79]:
os.chdir('T://lab_research/RES-Folder-UPOD/Echo_label/E_ResearchData/2_ResearchData')

In [80]:
labeled_documents = pd.read_json(f"./echo_doc_labels/{Class}.jsonl", lines=True)

In [81]:
labeled_documents.label.value_counts()

label
No label    3474
Mild         632
Normal       521
Moderate     243
Severe       130
Name: count, dtype: int64

In [82]:
Target_remapper = {'No label': 'No label', 
                   'Normal': 'Normal',
                   'Mild': 'Not normal',
                   'Moderate': 'Not normal',
                   'Severe': 'Not normal',
                   'Present': 'Not normal'
                   }
if reduce_labels:
    labeled_documents['label'] = labeled_documents.label.map(Target_remapper)
    print(labeled_documents.label.value_counts(dropna=False))

In [83]:
# Expand with label columns
Target_maps = {Label:i for i,Label in enumerate(labeled_documents['label'].unique())}

In [84]:
Target_maps

{'No label': 0, 'Normal': 1, 'Mild': 2, 'Severe': 3, 'Moderate': 4}

In [85]:
# Load the train/test hashes
test_hashes = pd.read_csv('./test_echoid.csv', sep=',')
train_hashes = pd.read_csv('./train_echoid.csv', sep=',')
print(train_hashes.columns)

print(f"Train hashes: {train_hashes.input_hash.nunique()}")
print(f"Test hashes: {test_hashes.input_hash.nunique()}")

Index(['ECHO_StudyID', 'ECHO_StudyID.1', 'input_hash', 'task_hash'], dtype='object')
Train hashes: 96026
Test hashes: 24051


In [86]:
print(Target_maps.keys())

dict_keys(['No label', 'Normal', 'Mild', 'Severe', 'Moderate'])


In [87]:
# We now make DataSets (a special HuggingFace structure)
# assuming cross-validation

DF = labeled_documents
DF.columns = ['sentence', 'labels', '_input_hash']

label2id = Target_maps
id2label = {v:k for k,v in label2id.items()}
num_labels = len(label2id)
DF['labels'] = DF['labels'].map(label2id)


if filter_reports:
    DF = DF.assign(sentence = echo_utils.report_filter(DF.sentence, 
                                            flag_terms=FLAG_TERMS, 
                                            save_terms=SAVE_TERMS)[0])
    DF = DF.loc[DF.sentence.notna()]

if deabbreviate:
    DeAbber = deabber.deabber(model_type='sbert', 
                              abbreviations=ABBREVIATIONS['nl']['echocardiogram'], 
                              min_sim=0.5, top_k=10)
    DF = DF.assign(sentence=DeAbber.deabb(DF.sentence.values, TokenRadius=3))


In [88]:
if use_multilabel:
    _multilabels = {label2id[k]: [label2id[l] for l in v]
                    for k,v in MULTILABELS.items()}
else:
    _multilabels = None

In [89]:

# TODO: make proper
DFtrain = DF.loc[DF._input_hash.isin(train_hashes.input_hash), ['sentence', 'labels']]
DFtrain_SAMPLED = DFtrain.groupby('labels').sample(NUM_SAMPLES, replace=True)
num_classes = DFtrain.labels.nunique()

DFtest = DF.loc[DF._input_hash.isin(test_hashes.input_hash), ['sentence', 'labels']]

print("Train labels:")
print(DFtrain.labels.value_counts())
print("Train labels sampled:")
print(DFtrain_SAMPLED.labels.value_counts())
print("Test labels:")
print(DFtest.labels.value_counts())

TrainSet = Dataset.from_pandas(DFtrain_SAMPLED)
TestSet = Dataset.from_pandas(DFtest)

HF_DataSet = DatasetDict(
    {'train' : TrainSet,
     'test': TestSet,
    }
)

Train labels:
labels
0    2694
2     501
1     419
4     191
3     104
Name: count, dtype: int64
Train labels sampled:
labels
0    500
1    500
2    500
3    500
4    500
Name: count, dtype: int64
Test labels:
labels
0    682
1    105
2    105
4     44
3     22
Name: count, dtype: int64


In [90]:
base_dir = "L://laupodteam/AIOS/Bram/language_modeling/Models/language_models/setfit"
logging_dir = os.path.join(base_dir, "setfit_logs")
setfit_output = os.path.join(base_dir, "./setfit_output")

os.makedirs(logging_dir, exist_ok=True)
os.makedirs(setfit_output, exist_ok=True)

train_args = TrainingArguments(logging_dir=logging_dir,
                               output_dir=setfit_output, 
                               evaluation_strategy='no',
                               num_iterations=6,
                               num_epochs=num_epochs,
                               body_learning_rate = (2e-5, 1e-5),
                               head_learning_rate= (1e-3),
                               batch_size=32,
                               seed=42,                                                          
                               loss=CosineSimilarityLoss,                               
                               save_strategy="epoch",
                               load_best_model_at_end=False,
                               show_progress_bar=True,)

In [91]:
model_body = SentenceTransformer(body)
model_head = SVC()
model_setfit = SetFitModel(model_body, model_head)
                                           
# use_differentiable_head=True, 
# head_params={"out_features": num_classes})

c:\Users\bes3\AppData\Local\pypoetry\Cache\virtualenvs\echolabeler-DyKQcQCO-py3.10\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Training setup

In [92]:
setfit_trainer = Trainer(model_setfit, 
                        train_dataset=TrainSet, 
                        eval_dataset=TestSet,
                        args = train_args,   
                        metric = compute_metrics,                         
                        column_mapping={"sentence": "text", 
                                        "labels": "label"})

Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

## run Training

In [93]:
setfit_trainer.train(train_args)

***** Running training *****
  Num unique pairs = 30000
  Batch size = 32
  Num epochs = 3
  Total optimization steps = 2814


  0%|          | 0/2814 [00:00<?, ?it/s]

  0%|          | 0/2814 [00:00<?, ?it/s]

{'embedding_loss': 0.2754, 'learning_rate': 7.092198581560284e-08, 'epoch': 0.0}
{'embedding_loss': 0.2749, 'learning_rate': 3.5460992907801423e-06, 'epoch': 0.05}


In [ ]:
def compute_metrics(predictions, labels):
  #logits, labels = logits_and_labels
  #predictions = np.argmax(logits, axis=-1)
  
  f1_micro = f1_score(labels, predictions, average = 'micro')
  f1_macro = f1_score(labels, predictions, average = 'macro')
  
  precision_micro = precision_score(labels, predictions, average = 'micro')
  precision_macro = precision_score(labels, predictions, average = 'macro')

  recall_micro = recall_score(labels, predictions, average = 'micro')
  recall_macro = recall_score(labels, predictions, average = 'macro')
  
  return {'precision_micro': precision_micro, 'precision_macro': precision_macro,
          'f1_score_micro': f1_micro, 'f1_score_macro': f1_macro, 
          'recall_micro': recall_micro, 'recall_macro': recall_macro}

In [ ]:
setfit_trainer.metric = compute_metrics
metrics = setfit_trainer.evaluate()

***** Running evaluation *****


In [ ]:
metrics

{'precision_micro': 0.8402922755741128,
 'precision_macro': 0.6018548071135666,
 'f1_score_micro': 0.8402922755741128,
 'f1_score_macro': 0.6613681103796277,
 'recall_micro': 0.8402922755741128,
 'recall_macro': 0.7740217403181765}

In [ ]:
robert_res = dict()
robert_res['aortic regurgitation'] = \
    {'precision_micro': 0.9303621169916435,
    'precision_macro': 0.8428764056416101,
    'f1_score_micro': 0.9303621169916435,
    'f1_score_macro': 0.8587103420346984,
    'recall_micro': 0.9303621169916435,
    'recall_macro': 0.8807387791045382
    }

robert_res['aortic stenosis'] =  \
   {'precision_micro': 0.8935281837160751,
 'precision_macro': 0.6898065746737378,
 'f1_score_micro': 0.8935281837160751,
 'f1_score_macro': 0.7744423670865763,
 'recall_micro': 0.8935281837160751,
 'recall_macro': 0.9305292256686146}


robert_res['diastolic_dysfunction'] =  \

In [ ]:
biolord_res = dict()
biolord_res['aortic regurgitation'] = \
{'precision_micro': 0.8180129990714949,
 'precision_macro': 0.6544492741142449,
 'f1_score_micro': 0.8180129990714949,
 'f1_score_macro': 0.6814811218681188,
 'recall_micro': 0.8180129990714949,
 'recall_macro': 0.7195548322882586
 }

biolord_res['aortic stenosis'] = \
{'precision_micro': 0.8402922755741128,
 'precision_macro': 0.6018548071135666,
 'f1_score_micro': 0.8402922755741128,
 'f1_score_macro': 0.6613681103796277,
 'recall_micro': 0.8402922755741128,
 'recall_macro': 0.7740217403181765}

biolord_res['diastolic_dysfunction'] = \

SyntaxError: invalid syntax (48850072.py, line 11)